In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn.functional as F
import cv2
from PIL import Image


In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/hamiltonian_graph/train_info.csv')

data = pd.Series(data.hamiltonian.values, index=data.file_name).to_dict()

folder = "/content/drive/MyDrive/hamiltonian_graph/train_images/"

folder_no = "/content/drive/MyDrive/hamiltonian_graph/labels/0/"
folder_yes = "/content/drive/MyDrive/hamiltonian_graph/labels/1/"


for name in os.listdir(folder):
  img = Image.open(os.path.join(folder, name))

  class_name = data.get(name)

  if class_name == 'yes':
    img.save(os.path.join(folder_yes, name), 'png')

  else:
    img.save(os.path.join(folder_no, name), 'png')


In [ ]:

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = ImageFolder(root='/content/drive/MyDrive/hamiltonian_graph/labels', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

class GraphClassifierCNN(nn.Module):
    def __init__(self):
        super(GraphClassifierCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

model = GraphClassifierCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        labels = labels.float().view(-1, 1)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")




Epoch 1/10, Loss: 0.579815019806512
Epoch 2/10, Loss: 0.32555683889835
Epoch 3/10, Loss: 0.2439213749155295
Epoch 4/10, Loss: 0.17797040716885662
Epoch 5/10, Loss: 0.104413159339012
Epoch 6/10, Loss: 0.06110494430243915
Epoch 7/10, Loss: 0.054014586176884045
Epoch 8/10, Loss: 0.03585629320532593
Epoch 9/10, Loss: 0.00724655811205044
Epoch 10/10, Loss: 0.0026263466113423486


In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd


folder = "/content/drive/MyDrive/hamiltonian_graph/test_images/"

new_folder = "/content/drive/MyDrive/hamiltonian_graph/all_test/all/"

for name in os.listdir(folder):
  img = Image.open(os.path.join(folder, name))

  class_name = data.get(name)

  img.save(os.path.join(new_folder, name), 'png')



In [ ]:
model.eval()

test_data = ImageFolder(root='/content/drive/MyDrive/hamiltonian_graph/all_test/', transform=transform)
test_loader = DataLoader(test_data, shuffle=False)

f = open("test_y", "w")
with torch.no_grad():
    for i, (images, labels) in enumerate(test_loader):
        outputs = model(images)
        predicted = (outputs > 0.5).int()
        sample_fname, _ = test_loader.dataset.samples[i]
        sample_fname = sample_fname[54:]
        f.write("{},{}\n".format(sample_fname, predicted.item()))
f.close()



In [ ]:
import csv

with open('test_y', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('submission.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('file_name', 'hamiltonian'))
        writer.writerows(lines)

In [ ]:
data = pd.read_csv('submission.csv')


submission = data
submission

,file_name,hamiltonian
0,graph1000.png,0
1,graph1001.png,0
2,graph1004.png,0
3,graph1008.png,0
4,graph1009.png,1
...,...,...
1567,graph987.png,0
1568,graph989.png,1
1569,graph991.png,1
1570,graph996.png,0
